# Daten aus der API von Chess.com
Ursprünglich habe ich über die API die monatlichen Datenstände als PGN-Dateien heruntergeladen, diese zu einer JSON-Datei zusammengeführt und  dann eingelesen und ausgewertet.
Dies erfordert lokalen Speicher. Da ich aber die App öffentlich deployen will, geht das so nicht. Ich habe kläglich versucht dies über einen S3-Bucket von AWS zu lösen aber habe das nicht hinbekommen.

#### Fehlende Spalten aus Ansatz 1.0: 
Start-Datum, Dauer in Minuten, Anzahl Züge, ECO, Spielzüge

## 1. Runterladen und Speichern als Dataframe

In [2]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
import urllib

In [3]:
text_input = "oezguen"
username = text_input.lower() # wird aus Streamlit gezogen

In [4]:
def datenziehung(username):
        # Use of the API to download monthly archives
        baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"

        # read the archives url and store in a list
        f = urllib.request.urlopen(baseUrl + "archives")
        archives = f.read().decode("utf-8")
        archives = archives.replace("{\"archives\":[\"", "\",\"")
        archivesList = archives.split("\",\"" + baseUrl)
        archivesList[len(archivesList)-1] = archivesList[len(archivesList)-1].rstrip("\"]}")
        df_list = []
        for i in range(len(archivesList)-1):
                url = baseUrl + archivesList[i+1]
                # Make an HTTP request to the URL
                response = requests.get(url)

                # Parse the response into a Python dictionary
                data = json.loads(response.text)

                # Convert the dictionary into a list of records
                records = data['games']

                # Convert the list of records into a dataframe and append it to the list
                df_list.append(pd.DataFrame.from_records(records))

        # Concatenate all the dataframes into a single dataframe
        df = pd.concat(df_list) 
        return df

In [5]:
df = datenziehung(username)

## 2. Datenbearbeitung

In [6]:
def datenbearbeitung(df):
    # Spalten-Titel umbennen
    dict = {'url': 'Link',
            'pgn': 'PGN',
            'time_control': 'TimeControl',
            'end_time': 'EndTime',
            'rated':'Bewertet',
            'start_time':'StartTime'
            }

    df.rename(columns=dict,
            inplace=True)

    df['meine Farbe'] = np.where(
        df['white'].apply(lambda x: x.get('username')) == text_input, 
        'Weiß', 
        'Schwarz'
        )

    df['mein Elo'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['white'].apply(lambda x: x.get('rating')), 
        df['black'].apply(lambda x: x.get('rating'))
        )

    df['Gegner Elo'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('rating')), 
        df['white'].apply(lambda x: x.get('rating'))
        )

    df['Ergebnis'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['white'].apply(lambda x: x.get('result')),
        df['black'].apply(lambda x: x.get('result'))
    )

    df['Gegner Ergebnis'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('result')),
        df['white'].apply(lambda x: x.get('result'))
    )

    df['Weiß Ergebnis'] = df['white'].apply(lambda x: x.get('result')) #drinlassen für Kontrolle

    df['Schwarz Ergebnis'] = df['black'].apply(lambda x: x.get('result')) #drinlassen für Kontrolle

    df['Gegner-Name'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('username')),
        df['white'].apply(lambda x: x.get('username'))
    )

    df['Gegner API'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('@id')),
        df['white'].apply(lambda x: x.get('@id'))
    )

    game_type_mapping = {
        "rapid":"Schnellschach",
        "bullet":"Bullet",
        "blitz":"Blitz",
        "daily":"Täglich",
        
    }
    df['Spiel-Art'] = df['time_class'].astype(str).map(game_type_mapping).astype('category')

    time_control_mapping = {
        "600":"10 Minuten",
        "60":"1 Minute",
        "60+1":"1 | 1",
        "300":"5 Minuten",
        "1/172800":"2 Tage",
        "180":"3 Minuten",
        "1800":"30 Minuten",
        "120+1":"2 | 1",
        "900+5":"10 | 5",
        "900":"15 Minuten",
        "300+5":"5 | 5",
        "180+2":"3 | 2",
        "30":"30 Sek.",
        "900+10":"10 | 10",
        "1/86400":"1 Tag"
    }	
    df['Zeit']= df['TimeControl'].astype(str).map(time_control_mapping)

    df['End-Datum'] = df['EndTime'].apply(datetime.datetime.fromtimestamp)
    df['Datum'] = df['End-Datum'].apply(lambda x: x.date())
    df['Uhrzeit'] = df['End-Datum'].apply(lambda x: x.time())
    
    df['Meine Genauigkeit'] = np.where(
        df.accuracies.isna(),
        {},
        df.accuracies
    )

    df['Meine Genauigkeit'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['Meine Genauigkeit'].apply(lambda x: x.get('white')),
        df['Meine Genauigkeit'].apply(lambda x: x.get('black'))   
    )

    df['Meine Genauigkeit'] =["{:.2%}".format(x/100) for x in df['Meine Genauigkeit']]

    df['Gegner Genauigkeit'] = np.where(
        df.accuracies.isna(),
        {},
        df.accuracies
    )

    df['Gegner Genauigkeit'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['Gegner Genauigkeit'].apply(lambda x: x.get('black')),
        df['Gegner Genauigkeit'].apply(lambda x: x.get('white'))   
    )
    
    df['Gegner Genauigkeit'] =["{:.2%}".format(x/100) for x in df['Gegner Genauigkeit']]

    ergebnis_mapping = {
        "checkmated":"Schachmatt",
        "timeout":"Zeitüberschreitung",
        "resigned":"Aufgabe",
        "win":"gewonnen",
        "stalemate":"Patt",
        "abandoned":"Verlassen",
        "repetition":"Stellungswiederholung",
        "timevsinsufficient":"Zeitüberschreitung gegen unzureichendes Material",
        "threecheck":"3 Mal Schach-gestellt",
        "agreed":"Vereinbarung",
        "insufficient":"Nicht genügend Material",
        "50move":"50 Züge Regel",
        "lose":"verloren",
        "kingofthehill":"Gegnerischer König hat die Mitte erreicht",
        "bughousepartnerlose":"Bughouse-Partner hat verloren"
    }	

    df['Ausgang'] = np.where(
        df['Ergebnis'] == 'win',
        'gewonnen',
        np.where(
            df['Gegner Ergebnis'] == 'win',
            'veloren',
            'unentschieden'
        )
    )

    df['Ausgang-Grund'] = np.where(
        df['Ergebnis'] == 'win',
        df['Gegner Ergebnis'].astype(str).map(ergebnis_mapping),
        df['Ergebnis'].astype(str).map(ergebnis_mapping)
    )


    ergebnis_mapping = {
        "chess":"klassisch",
        "kingofthehill":"King of the Hill",
        "threecheck":"3 Schach Schach",
        "chess960":"Schach960",
        "crazyhouse":"Einsetzschach",
        "bughouse":"Tandemschach",
        "oddschess":"Odds Chess"
    }	
    df['Variante'] = df.rules.map(ergebnis_mapping)

    time_control_mapping = {
        "10":"10 Sek.",
        "30":"30 Sek.",
        "60":"1 Minute",
        "60+1":"1 | 1",
        "120+1":"2 | 1",

        "180":"3 Minuten",
        "180+1":"3 | 1",
        "180+2":"3 | 2",
        "180+3":"3 | 3",
        "180+5":"3 | 5",
        "300":"5 Minuten",
        "300+1":"5 | 1",
        "300+2":"5 | 2",
        "300+3":"5 | 3",
        "300+5":"5 | 5",
        "300+10":"5 | 10",
        "420":"7 Minuten",
        "480":"8 Minuten",
        "480+2":"8 | 2",

        "600":"10 Minuten",
        "600+2":"10 | 2",
        "600+3":"10 | 3",
        "600+5":"10 | 5",
        "600+10":"10 | 10",
        "900":"15 Minuten",
        "900+2":"15 | 2",
        "900+3":"15 | 3",
        "900+5":"15 | 5",
        "900+10":"15 | 10",
        "1200":"20 Minuten",
        "1200+10":"20 | 10",
        "1800":"30 Minuten",
        "2700+15":"45 | 15",
        "3600+45":"60 | 45",

        "1/86400":"1 Tag",
        "1/172800":"2 Tage",
        "1/259200":"3 Tage",
        "1/432000":"5 Tage",
        "1/604800":"7 Tage",
        "1/1209600":"14 Tage"
    }	
    
    df['Zeit']= df['TimeControl'].astype(str).map(time_control_mapping)

    wochentag_mapping = {
        0:"Montag",
        1:"Dienstag",
        2:"Mittwoch",
        3:"Donnerstag",
        4:"Freitag",
        5:"Samstag",
        6:"Sonntag",
        7:"2 | 1"
    }	
    
    df['Wochentag']= df['End-Datum'].apply(lambda time: time.dayofweek).map(wochentag_mapping)


    df = df[[
        'Datum','Uhrzeit','Spiel-Art', 'Zeit', 'Variante', 'Bewertet',
        'Ausgang', 'Ausgang-Grund',
        'meine Farbe', 'mein Elo', 'Gegner Elo', 'Gegner-Name',
        'Meine Genauigkeit', 'Gegner Genauigkeit', 'Link'
    ]]

    return df

In [7]:
df = datenbearbeitung(df)

In [8]:
df

,Datum,Uhrzeit,Spiel-Art,Zeit,Variante,Bewertet,Ausgang,Ausgang-Grund,meine Farbe,mein Elo,Gegner Elo,Gegner-Name,Meine Genauigkeit,Gegner Genauigkeit,Link
0,2017-01-11,21:58:46,Blitz,10 Minuten,klassisch,True,veloren,Schachmatt,Weiß,1038,1362,BuBasha115,nan%,nan%,https://www.chess.com/game/live/1897250091
1,2017-01-11,22:00:11,Blitz,10 Minuten,klassisch,True,veloren,Schachmatt,Schwarz,917,1100,Monsieurponeymagique,nan%,nan%,https://www.chess.com/game/live/1897262846
2,2017-01-23,15:30:13,Täglich,2 Tage,klassisch,True,veloren,Zeitüberschreitung,Schwarz,1200,1200,Thecolmie,nan%,nan%,https://www.chess.com/game/daily/155435810
0,2017-02-15,15:55:44,Schnellschach,15 | 10,klassisch,True,veloren,Schachmatt,Weiß,1082,1371,Danilofalp,nan%,nan%,https://www.chess.com/game/live/1952673922
1,2017-02-15,16:12:10,Schnellschach,15 | 10,klassisch,True,veloren,Aufgabe,Schwarz,961,1130,ruth_scorpion47474,nan%,nan%,https://www.chess.com/game/live/1952686228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2023-01-11,21:03:41,Schnellschach,10 Minuten,klassisch,True,veloren,Aufgabe,Schwarz,912,945,left97,nan%,nan%,https://www.chess.com/game/live/67206096387
65,2023-01-12,08:00:55,Schnellschach,10 Minuten,klassisch,True,gewonnen,Aufgabe,Schwarz,921,942,Bsakthivel,nan%,nan%,https://www.chess.com/game/live/67245578729
66,2023-01-12,08:15:01,Schnellschach,10 Minuten,klassisch,True,veloren,Aufgabe,Weiß,913,933,MacRed1010,nan%,nan%,https://www.chess.com/game/live/67245702599
67,2023-01-12,08:30:07,Schnellschach,10 Minuten,klassisch,True,gewonnen,Schachmatt,Weiß,920,867,Hung2k20,nan%,nan%,https://www.chess.com/game/live/67247401749


In [9]:
df.to_csv('meine_daten.csv')